In [2]:
import os
import pandas as pd
import subprocess
from glob import glob
from transnetv2 import TransNetV2
import cv2
import numpy as np

In [ ]:
def get_and_save_frame(video_path, frame_id, save_path, filename):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open the video file.")
        return
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    ret, frame = cap.read()
    if not ret:
        print(f"Error: Could not read frame {frame_id}.")
        cap.release()
        return
    os.makedirs(save_path, exist_ok=True)
    save_file_path = os.path.join(save_path, filename)
    cv2.imwrite(save_file_path, frame)
    print(f"Frame {frame_id} saved successfully at {save_file_path}.")
    cap.release()

In [3]:
os.makedirs('keyframes', exist_ok=True)
os.makedirs('video', exist_ok=True)
os.makedirs('map-keyframes', exist_ok=True)
model = TransNetV2()

[TransNetV2] Using weights from /Library/Python/3.9/site-packages/transnetv2-1.0.0-py3.9.egg/transnetv2/transnetv2-weights/.


AttributeError: module 'tensorflow' has no attribute 'saved_model'

In [24]:
links = pd.read_csv("link_down_video.csv")
links = links[links['Type'] == 'Video']
video_link_dict = {}

for index, row in links.iterrows():
    video_link_dict[row['Filename'][:-4]] = row['Direct Link']

video_link_dict

{'Videos_L01': 'https://atm249495-s3user.vcos.cloudstorage.com.vn/aic24-b5/Videos_L01.zip',
 'Videos_L02': 'https://atm249496-s3user.vcos.cloudstorage.com.vn/aic24-b6/Videos_L02.zip',
 'Videos_L03': 'https://atm249497-s3user.vcos.cloudstorage.com.vn/aic24-b7/Videos_L03.zip',
 'Videos_L04': 'https://atm249498-s3user.vcos.cloudstorage.com.vn/aic24-b8/Videos_L04.zip',
 'Videos_L05': 'https://atm249495-s3user.vcos.cloudstorage.com.vn/aic24-b5/Videos_L05.zip',
 'Videos_L06': 'https://atm249496-s3user.vcos.cloudstorage.com.vn/aic24-b6/Videos_L06.zip',
 'Videos_L07': 'https://atm249497-s3user.vcos.cloudstorage.com.vn/aic24-b7/Videos_L07.zip',
 'Videos_L08': 'https://atm249498-s3user.vcos.cloudstorage.com.vn/aic24-b8/Videos_L08.zip',
 'Videos_L09': 'https://atm249495-s3user.vcos.cloudstorage.com.vn/aic24-b5/Videos_L09.zip',
 'Videos_L10': 'https://atm249496-s3user.vcos.cloudstorage.com.vn/aic24-b6/Videos_L10.zip',
 'Videos_L11': 'https://atm249497-s3user.vcos.cloudstorage.com.vn/aic24-b7/Video

In [29]:
for video, link in video_link_dict.items():
    # command = [
    #     'aria2c', link, 
    #     '--dir', 'video', 
    #     '-s', '16',
    #     '-x', '16', 
    # ]
    # subprocess.run(command, check=True)

    # command = ['unzip', f'{video}.zip', '-d', 'video']
    # subprocess.run(command, check=True)

    # os.rename('video/video', 'video/{video}')

    L = video[-3:]
    os.makedirs(f'keyframes/keyframes_{L}', exist_ok=True)

    videos = glob(os.getcwd() + '/' + 'video/{video}/*.mp4')
    videos.sort()
    for sub_video in videos:
        with open(f'map-keyframes/{sub_video}.csv', 'w') as file:
                file.write('n,pts_time,fps,frame_idx')
        video_frames, single_frame_predictions, all_frame_predictions = \
            model.predict_video("sub_video")
        frame_interval = model.predictions_to_scenes(single_frame_predictions)
        for i, row in enumerate(frame_interval):
            frame_id_extracted = int((row[0] + row[1])/2)
            get_and_save_frame(sub_video, frame_id_extracted, f'keyframes/keyframes_{L}', f'{(i+1):03d}.jpg')
            with open(f'map-keyframes/{sub_video}.csv', 'w') as file:
                file.write(f'{i+1},{round(frame_id_extracted / 25.0, 2)},25.0,{frame_id_extracted}')

['keyframes/keyframes_L10', 'keyframes/keyframes_L11', 'keyframes/keyframes_L05', 'keyframes/keyframes_L02', 'keyframes/keyframes_L03', 'keyframes/keyframes_L04', 'keyframes/keyframes_L12', 'keyframes/keyframes_L08', 'keyframes/keyframes_L01', 'keyframes/keyframes_L06', 'keyframes/keyframes_L07', 'keyframes/keyframes_L09']
['keyframes/keyframes_L10', 'keyframes/keyframes_L11', 'keyframes/keyframes_L05', 'keyframes/keyframes_L02', 'keyframes/keyframes_L03', 'keyframes/keyframes_L04', 'keyframes/keyframes_L12', 'keyframes/keyframes_L08', 'keyframes/keyframes_L01', 'keyframes/keyframes_L06', 'keyframes/keyframes_L07', 'keyframes/keyframes_L09']
['keyframes/keyframes_L10', 'keyframes/keyframes_L11', 'keyframes/keyframes_L05', 'keyframes/keyframes_L02', 'keyframes/keyframes_L03', 'keyframes/keyframes_L04', 'keyframes/keyframes_L12', 'keyframes/keyframes_L08', 'keyframes/keyframes_L01', 'keyframes/keyframes_L06', 'keyframes/keyframes_L07', 'keyframes/keyframes_L09']
['keyframes/keyframes_L10